# AI ügynökök építése tartós memóriával a Cognee segítségével

Ez a jegyzetfüzet bemutatja, hogyan lehet intelligens AI ügynököket létrehozni kifinomult memóriafunkciókkal a [**cognee**](https://www.cognee.ai/) használatával - egy nyílt forráskódú AI memória, amely ötvözi a tudásgrafikonokat, szemantikus keresést és munkamenet-kezelést, hogy kontextusérzékeny AI rendszereket hozzon létre.

## 🎯 Tanulási célok

A bemutató végére megérted, hogyan kell:
- **Tudásgrafikonokat építeni beágyazásokkal támogatva**: Átalakítani a strukturálatlan szöveget strukturált, lekérdezhető tudássá
- **Munkamenet-memóriát megvalósítani**: Többfordulós beszélgetéseket létrehozni automatikus kontextusmegőrzéssel
- **Beszélgetéseket tartósítani**: Opcionálisan tárolni fontos interakciókat hosszú távú memóriában későbbi hivatkozás céljából
- **Természetes nyelvű lekérdezést használni**: Hozzáférni és kihasználni a történelmi kontextust új beszélgetésekben
- **Memóriát vizualizálni**: Felfedezni az ügynök tudásgrafikonjának kapcsolatait


## 🏗️ Mit fogsz építeni

Ebben az útmutatóban egy **Kódolási Asszisztenst** hozunk létre tartós memóriával, amely:

### 1. **Tudásbázis létrehozása**
   - Fejlesztői profil és szakértelem információkat dolgoz fel
   - Feldolgozza a Python programozási alapelveket és legjobb gyakorlatokat
   - Tárolja a fejlesztők és AI asszisztensek közötti korábbi beszélgetéseket

### 2. **Munkamenet-tudatos beszélgetések**
   - Fenntartja a kontextust ugyanazon munkamenet több kérdése között
   - Automatikusan gyorsítótárazza az egyes kérdés/válasz párokat a hatékony visszakeresés érdekében
   - Koherens, kontextusfüggő válaszokat ad a beszélgetési előzmények alapján

### 3. **Hosszú távú memória**
   - Fontos beszélgetéseket tartós memóriába ment
   - Releváns emlékeket hív elő a tudásbázisból és korábbi munkamenetekből az új interakciókhoz
   - Egyre bővülő tudásbázist épít, amely idővel javul

### 4. **Intelligens memória-visszakeresés**
   - Grafikon-tudatos szemantikus keresést használ a tárolt tudás releváns információinak megtalálásához
   - Szűri a kereséseket adatalcsoportok szerint (fejlesztői információk vs. alapelvek)
   - Több adatforrást kombinál, hogy átfogó válaszokat nyújtson


## 📋 Előfeltételek és beállítás

### Rendszerkövetelmények

Mielőtt elkezdenéd, győződj meg arról, hogy rendelkezel az alábbiakkal:

1. **Python környezet**
   - Python 3.9 vagy újabb
   - Virtuális környezet (ajánlott)
   
2. **Redis Cache** (Szükséges a munkamenet-kezeléshez)
   - Helyi Redis: `docker run -d -p 6379:6379 redis`
   - Vagy használj egy kezelt Redis szolgáltatást
   
3. **LLM API hozzáférés**
   - OpenAI API kulcs vagy más szolgáltatók (lásd a [dokumentációt](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Adatbázis konfiguráció**
   - Alapértelmezés szerint nincs szükség konfigurációra. A Cognee fájlalapú adatbázisokat használ (LanceDB és Kuzu)
   - Opcionálisan beállíthatod az Azure AI Search-t vektortárolóként (lásd a [dokumentációt](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Környezeti konfiguráció

Hozz létre egy `.env` fájlt a projektkönyvtáradban az alábbi változókkal:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cognee memóriaarchitektúrájának megértése

### Hogyan működik a Cognee

A Cognee egy kifinomult memória rendszert biztosít, amely túlmutat az egyszerű kulcs-érték tároláson:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Főbb összetevők:

1. **Tudásgráf**: Entitásokat, kapcsolatokat és szemantikai összefüggéseket tárol
2. **Vektorembeddingek**: Lehetővé teszik a szemantikai keresést az összes tárolt információ között
3. **Munkamenet gyorsítótár**: Fenntartja a beszélgetési kontextust a munkamenetek során és között
4. **NodeSets**: Az adatokat logikai kategóriákba szervezi a célzott visszakeresés érdekében

### Memóriatípusok ebben az útmutatóban:

- **Tartós memória**: Hosszú távú tárolás a tudásgráfban
- **Munkamenet memória**: Ideiglenes beszélgetési kontextus a Redis gyorsítótárban
- **Szemantikai memória**: Vektor-alapú hasonlósági keresés az összes adat között


## 📦 Szükséges csomagok telepítése

Telepítsd a Cognee-t Redis támogatással a munkamenet-kezeléshez:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Környezet inicializálása és könyvtárak betöltése

Győződjön meg róla, hogy:
1. A Redis fut (pl. Dockerrel: `docker run -d -p 6379:6379 redis`)
2. A környezeti változók be vannak állítva, mielőtt a cache modulokat importálná
3. Ha szükséges, indítsa újra a kernelt, és futtassa a cellákat sorrendben

A következő cella:
1. Betölti a környezeti változókat a `.env` fájlból
2. Beállítja a Cognee-t az LLM beállításaival
3. Engedélyezi a gyorsítótárazást a munkamenet-kezeléshez
4. Ellenőrzi, hogy minden komponens megfelelően csatlakozik-e


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Tárhely könyvtárak konfigurálása

A Cognee két külön könyvtárat használ a működéséhez:
- **Adatgyökér**: Az importált dokumentumokat és a feldolgozott adatokat tárolja
- **Rendszergyökér**: A tudásgráf adatbázist és a rendszer metaadatait tartalmazza

Ebben az útmutatóban elkülönített könyvtárakat fogunk létrehozni az alábbiak szerint:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Memóriaállapot visszaállítása

Mielőtt elkezdenénk a memória rendszerünk kiépítését, győződjünk meg róla, hogy tiszta lappal indulunk.

> 💡 **Tipp**: Ezt a lépést kihagyhatod, ha meg szeretnéd őrizni a korábbi futtatásokból származó meglévő emlékeket, amikor később használod ezt a jegyzetfüzetet.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 1. rész: A tudásbázis felépítése

### Adatforrások a fejlesztői asszisztensünkhöz

Háromféle adatot fogunk feldolgozni, hogy átfogó tudásbázist hozzunk létre:

1. **Fejlesztői profil**: Személyes szakértelem és technikai háttér
2. **Python legjobb gyakorlatok**: A Python filozófiája gyakorlati iránymutatásokkal
3. **Korábbi beszélgetések**: Fejlesztők és AI asszisztensek közötti korábbi kérdés-válasz ülések

Ez a sokszínű adat lehetővé teszi, hogy az ügynökünk:
- Megértse a felhasználó technikai kontextusát
- Alkalmazza a legjobb gyakorlatokat az ajánlásokban
- Tanuljon a korábbi sikeres interakciókból


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Adatok feldolgozása tudásgráffá

Most a nyers szöveget strukturált memóriává alakítjuk. Ez a folyamat:

1. **Adatok hozzáadása a NodeSetekhez**: Az információk logikai kategóriákba rendezése
   - `developer_data`: Fejlesztői profil és beszélgetések
   - `principles_data`: Python legjobb gyakorlatok és irányelvek

2. **Cognify Pipeline futtatása**: Entitások és kapcsolatok kinyerése, valamint beágyazások létrehozása
   - Kulcsfogalmak azonosítása
   - Szemantikus kapcsolatok létrehozása az összefüggő információk között
   - Vektoros beágyazások generálása

Ez néhány pillanatot igénybe vehet, mivel az LLM feldolgozza a szöveget és felépíti a gráfstruktúrát:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizáljuk a Tudásgráfot

Fedezzük fel a tudásgráfunk szerkezetét. A vizualizáció a következőket mutatja:
- **Csomópontok**: A szövegből kinyert entitások (fogalmak, technológiák, személyek)
- **Élek**: Kapcsolatok és összefüggések az entitások között
- **Klaszterek**: Kapcsolódó fogalmak csoportosítva szemantikai hasonlóság alapján

Nyisd meg a generált HTML fájlt a böngésződben, hogy interaktívan felfedezhesd a gráfot:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Memória gazdagítása a Memify segítségével

A `memify()` függvény elemzi a tudásgráfot, és intelligens szabályokat generál az adatokkal kapcsolatban. Ez a folyamat:
- Azonosítja a mintákat és a bevált gyakorlatokat
- Cselekvésre alkalmas irányelveket hoz létre a tartalom alapján
- Kapcsolatokat alakít ki a különböző tudásterületek között

Ezek a szabályok segítik az ügynököt abban, hogy megalapozottabb döntéseket hozzon a kérdések megválaszolásakor. Egy második vizualizáció rögzítése segít összehasonlítani, hogyan sűrűsödik a gráf a gazdagítás után.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 2. rész: Intelligens memória-visszakeresés

### 1. bemutató: Tudás integrálása több dokumentumból

Most, hogy elkészült a tudásgráfunk, teszteljük, hogyan kombinálja a Cognee az információkat több forrásból, hogy összetett kérdésekre válaszoljon.

Az első lekérdezés bemutatja:
- **Szemantikai megértés**: Releváns fogalmak megtalálása akkor is, ha azok nincsenek kifejezetten megemlítve
- **Kereszthivatkozás**: Fejlesztői profil és Python alapelvek kombinálása
- **Kontekstuális érvelés**: Legjobb gyakorlatok alkalmazása konkrét megvalósításokra

### 2. bemutató: Szűrt keresés NodeSetekkel

A második lekérdezés megmutatja, hogyan célozhatunk meg a tudásgráf specifikus részhalmazait:
- A `node_name` paramétert használja, hogy csak a `principles_data`-ban keressen
- Koncentrált válaszokat ad egy adott tudásterületről
- Hasznos, ha területspecifikus információra van szükség


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 3. rész: Munkamenet-kezelés beállítása

### Beszélgetési memória engedélyezése

A munkamenet-kezelés kulcsfontosságú a kontextus fenntartásához több interakció során. Itt a következőket fogjuk elvégezni:

1. **Felhasználói kontextus inicializálása**: Hozzon létre vagy hívjon elő egy felhasználói profilt a munkamenet nyomon követéséhez
2. **Gyorsítótár-motor konfigurálása**: Csatlakozzon a Redis-hez a beszélgetési előzmények tárolásához
3. **Munkamenet-változók engedélyezése**: Állítson be olyan kontextusváltozókat, amelyek megmaradnak a lekérdezések között

> ⚠️ **Fontos**: Ehhez szükséges, hogy a Redis fusson, és a környezetben a `CACHING=true` legyen beállítva


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Segítő funkció: Munkamenet előzményeinek megtekintése

Ez a segédfunkció lehetővé teszi, hogy megvizsgáljuk a Redis-ben tárolt beszélgetési előzményeket. Hasznos a következőkre:
- Munkamenet-kezelés hibakeresése
- Annak ellenőrzése, hogy a beszélgetések tárolva vannak-e
- Annak megértése, hogy milyen kontextus áll rendelkezésre az ügynök számára


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## 1. szekció: Async Support Lab — Első kérdés

Indítsd el az `async-support-lab` szekciót azzal, hogy megkérdezed, milyen telemetria-barát asyncio minták alkalmazhatók egy hatalmas webes adatgyűjtőhöz. A grafikon már ismeri az asyncio-t, az aiohttp-t és a monitorozási gyakorlatokat, így a válasznak tükröznie kell a korábbi beszélgetéseket, miközben az új kérdéshez igazodik.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Az 1. munkamenet memóriájának ellenőrzése az első csere után

A `show_history(session_1)` futtatása közvetlenül az első kérdés után megerősíti, hogy a Cognee mind a promptot, mind a választ beírta a Redisbe. Egy bejegyzést kell látnod a párhuzamossági útmutatóval.


In [ ]:
await show_history(session_1)

## 1. szekció: Adatmodellek nyomon követése

Ezután azt kérdezzük: "Mikor válasszak dataclasses-t a Pydantic helyett?" ugyanazzal a munkamenet azonosítóval. A Cognee-nek össze kell kapcsolnia a Python alapelveket és a korábbi FastAPI beszélgetéseket, hogy árnyalt tanácsot adjon—bemutatva, hogy a kontextus áthúzódik egy megnevezett munkameneten belül.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Ellenőrizze, hogy az 1. munkamenet előzményei tartalmazzák-e mindkét fordulót

Egy újabb `show_history(session_1)` hívásnak két kérdés-válasz bejegyzést kell mutatnia. Ez megfelel a Mem0 labor "memória visszajátszás" lépésének, és bizonyítja, hogy a további fordulók ugyanahhoz a jegyzőkönyvhöz kapcsolódnak.


In [ ]:
await show_history(session_1)

## 2. szekció: Tervezési áttekintő szál — Új szekció

Annak érdekében, hogy elkülönítsük a szálakat, elindítjuk a `design-review-session`-t, és naplózási útmutatást kérünk az incidensek áttekintéséhez. Bár az alapul szolgáló tudásbázis ugyanaz, az új szekcióazonosító külön tartja a jegyzőkönyveket.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## 2. munkamenet áttekintése - Történelem

A `show_history(session_2)` csak a tervezési áttekintéshez tartozó kérdés-válasz párost kell, hogy megjelenítse. Hasonlítsd össze az 1. munkamenettel, hogy kiemeld, hogyan tartja a Cognee az önálló átiratokat, miközben újrahasznosítja a megosztott tudásgráfot.


In [ ]:
await show_history(session_2)

## Összefoglaló

Gratulálunk! Mostantól a kódolási asszisztensed egy valódi hosszú távú memória réteggel rendelkezik, amelyet a Cognee biztosít.

Ebben az útmutatóban nyers fejlesztői tartalmat (kód, dokumentációk, csevegések) alakítottál át egy gráf + vektormemóriává, amelyet az ügynököd kereshet, elemezhet és folyamatosan fejleszthet.

Amit megtanultál:

1. **Nyers szövegből AI memóriává**: Hogyan dolgozza fel a Cognee a strukturálatlan adatokat, és alakítja intelligens, kereshető memóriává egy kombinált vektor + tudásgráf architektúra segítségével.

2. **Gráf gazdagítása memify-val**: Hogyan léphetsz túl az alapvető gráf létrehozáson, és használhatod a memify-t származtatott tények és gazdagabb kapcsolatok hozzáadására a meglévő gráfhoz.

3. **Többféle keresési stratégia**: Hogyan lehet memóriát lekérdezni különböző keresési típusokkal (gráf-tudatos kérdezz-felelek, RAG-stílusú kiegészítés, betekintések, nyers szövegrészletek, kódkeresés stb.) az ügynök igényei szerint.

4. **Vizualizációs felfedezés**: Hogyan lehet ellenőrizni és hibakeresni, amit a Cognee létrehozott gráf vizualizációk és a Cognee UI segítségével, hogy ténylegesen láthasd, hogyan van strukturálva a tudás.

5. **Munkamenet-tudatos memória**: Hogyan lehet kombinálni a munkamenetenkénti kontextust a tartós szemantikai memóriával, hogy az ügynökök emlékezni tudjanak a futások között anélkül, hogy információt szivárogtatnának a felhasználók között.


## Főbb tanulságok
1. Memória mint Tudásgráf beágyazásokkal támogatva

    - **Strukturált megértés**: A Cognee egyesíti a vektortárat és a gráftárat, így az adataid nemcsak jelentés alapján kereshetők, hanem kapcsolatok által is összekötöttek. A Cognee alapértelmezés szerint fájlalapú adatbázisokat használ (LanceDB a vektortárhoz, Kuzu a gráfadatbázishoz).

    - **Kapcsolat-tudatos visszakeresés**: A válaszok nemcsak „hasonló szövegeken” alapulhatnak, hanem azon is, hogy az entitások hogyan kapcsolódnak egymáshoz.

    - **Élő memória**: A memóriaréteg fejlődik, növekszik, és egy összekapcsolt gráfként lekérdezhető marad.

2. Keresési és érvelési módok
    - **Hibrid visszakeresés**: A keresés ötvözi a vektorszerű hasonlóságot, a gráfstruktúrát és az LLM érvelést, a nyers szövegrészletek keresésétől a gráf-tudatos kérdésmegoldásig.

    - **Illeszd a módot a feladathoz**: Használj kiegészítés-stílusú módokat, ha természetes nyelvű válaszokat szeretnél, és részlet/összefoglaló/gráf módokat, ha az ügynöködnek nyers kontextusra van szüksége vagy saját érvelését kell irányítania.

3. Személyre szabott, munkamenet-tudatos ügynökök
    - **Munkamenet-kontektsztus + hosszú távú memória**: A Cognee külön kezeli a rövid távú „szál” kontextust és a hosszú távú, felhasználói vagy szervezeti szintű memóriát.

## Valós alkalmazások

1. **Vertikális AI ügynökök**

    Használd ennek a jegyzetfüzetnek a mintáját, hogy domain-specifikus copilotákat hozz létre, amelyek a Cognee-t használják visszakeresési és érvelési magként:

- **Fejlesztői copilóták**: Kódellenőrzés, incidenselemzés és architektúra-asszisztensek, amelyek a kódot, API-kat, tervezési dokumentumokat és jegyeket egyetlen memóriagráfként kezelik.

- **Ügyfélközpontú copilóták**: Támogatási vagy sikerügynökök, amelyek termékdokumentumokból, GYIK-ből, CRM jegyzetekből és korábbi jegyekből merítenek gráf-tudatos visszakereséssel és hivatkozott válaszokkal.

- **Belső szakértői copilóták**: Politikai, jogi vagy biztonsági asszisztensek, amelyek összekapcsolt szabályok, irányelvek és történelmi döntések alapján érvelnek, nem pedig elszigetelt PDF-ek alapján.

    A Cognee kifejezetten tartós, pontos memóriaként van pozícionálva AI ügynökök számára, egy élő tudásgráfot biztosítva, amely az ügynök mögé illeszkedik, és helyettesíti az ad-hoc vektortárakat és egyedi gráfkódokat.

2. **Adatsilók egyesítése egy memóriává**

    Ugyanez a megközelítés segít egy egységes memóriaréteg létrehozásában szétszórt források között:

- **Silókból egy gráfba**: Strukturált (pl. adatbázisok) és strukturálatlan adatok (pl. dokumentumok, csevegések) betöltése egyetlen gráfba, amelyet beágyazások támogatnak, ahelyett, hogy minden rendszerhez külön indexeket használnánk.

- **Források közötti érvelés hivatkozásokkal**: Többlépéses érvelés futtatása minden felett—„csatlakoztasd” a naplókat, metrikákat és dokumentumokat a gráfon keresztül—és mégis megalapozott válaszokat adj vissza forrásmegjelöléssel.

- **Tudásközpontok**: Olyan területeken, mint a banki szektor vagy az oktatás, a Cognee már használatban van PDF-ek, belső rendszerek és alkalmazásadatok egyesítésére egy tudásgráfba vektorokkal, hogy az ügynökök pontos, hivatkozott kontextussal válaszolhassanak a kérdésekre.

## Következő lépések

Már megvalósítottad az alapvető memóriaciklust. Íme néhány természetes kiterjesztés, amelyet önállóan kipróbálhatsz (részletekért lásd a [Cognee dokumentációt](https://docs.cognee.ai/)):

1. **Kísérletezz az időbeli tudatossággal**: Kapcsold be az időbeli cognify-t, hogy eseményeket és időbélyegeket vonj ki a szövegből.

2. **Vezess be ontológia-alapú érvelést**: Határozz meg egy OWL ontológiát a területedhez. Használd a Cognee ontológia támogatását, hogy a kinyert entitások és kapcsolatok ebben a sémában legyenek rögzítve, javítva a gráf minőségét és a terület-specifikus válaszokat.

3. **Adj hozzá egy visszacsatolási hurkot**: Hagyd, hogy a Cognee a valós felhasználói visszajelzések alapján állítsa be a gráf élsúlyait, így a visszakeresés idővel javul, nem marad statikus.

4. **Finomhangolás személyre szabásra és munkamenet-viselkedésre**: Használj felhasználói azonosítókat, bérlőket és adatállományokat, hogy minden személy vagy csapat saját nézetet kapjon a megosztott memóriamotor felett.

5. **Skálázás összetettebb ügynökökre**: Csatlakoztasd a Cognee-t ügynökkeretrendszerekhez, hogy több ügynökből álló rendszereket építs, amelyek mind ugyanazt a memóriaréteget osztják meg. *Microsoft Agent Framework x Cognee plugin hamarosan érkezik.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Felelősség kizárása**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
